In [ ]:
pip install mysql-connector-python
pip install pandavro

In [ ]:
import pandas as pd
import numpy as np
import mysql.connector as mysql
from mysql.connector import Error
import os
import pandavro as pdx


In [ ]:
dir_path = r'C:\Users\gian.spoletini\Documents\DE Challenge\CSV'
avro_path = r'C:\Users\gian.spoletini\Documents\DE Challenge\AVRO'

In [ ]:
try:
    conn = mysql.connect(host='127.0.0.1', user='root',  
                        password='root')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("USE MIGRATION")
except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
file_paths = []
file_names = []
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        file_paths.append(path)
        file_names.append(os.path.splitext(path)[0])
print(file_paths)
print(file_names)

In [ ]:
for path in file_paths:
    avro_file = path.replace(".csv", ".avro")
    avro_complete_path = rf'{avro_path}\{avro_file}'
    try:
        df_avro = pdx.read_avro(avro_complete_path)
    except:
        pass 
    file_name = path.split(".")[0]
    
    
    for i,row in df_avro.iterrows():
        try:
            sql = "INSERT INTO Migration."+file_name+" VALUES ("+(df_csv.shape[1]*"%s,")[:-1]+")"
            print(sql)
            cursor.execute(sql, tuple(row))
            conn.commit()
        except Error as e:
            str(e).replace("'","")
            print(e)
            error_sql = "INSERT INTO Migration.error_log VALUES (1,'"+ str(file_name) +"' , "+ str(row[0])+" , '"+ str(e) +")"
            print (error_sql)  
            cursor.execute(error_sql, tuple(row))
            conn.commit()

    